# Парсинг статей с помощью библиотеки arxiv
Нас интересуют статьи, касающиеся NLP. Это категория cs.CL.

Инфа из arXiv Category Taxonomy:
> **cs.CL (Computation and Language)**
>
>  Covers natural language processing. Roughly includes material in ACM Subject Class I.2.7. Note that work on artificial languages (programming languages, logics, formal systems) that does not explicitly address natural-language issues broadly construed (natural-language processing, computational linguistics, speech, text retrieval, etc.) is not appropriate for this area.

In [1]:
import ssl
import certifi
import urllib.request

import arxiv
from pathlib import Path
from tqdm import tqdm

In [2]:
# Штука чтобы не было SSL Certificate error
ctx = ssl.create_default_context(cafile=certifi.where())

https_handler = urllib.request.HTTPSHandler(context=ctx)
opener = urllib.request.build_opener(https_handler)
urllib.request.install_opener(opener)

In [3]:
# ===== ПЕРЕМЕННЫЕ И ДИРЕКТОРИИ =====
N = 100 # Количество статей в поиске. После фильтрации по категории количество может уменьшиться
CAT = 'cs.CL' # Целевая категория, которую хотим парсить.

# Куда будем скачивать данные
DATA_DIR = Path('data')
DATA_DIR.mkdir(parents=True, exist_ok=True)

RAW_TEX_DIR = Path('data/raw/tex')
RAW_TEX_DIR.mkdir(parents=True, exist_ok=True)

RAW_PDF_DIR = Path('data/raw/pdf')
RAW_PDF_DIR.mkdir(parents=True, exist_ok=True)

In [4]:
client = arxiv.Client(
    page_size=100,
    delay_seconds=3,
    num_retries=3
)

In [5]:
# Ищем N статей с категорией cs.CL, сортирует по самым свежим статьям
# Важно: в поиске будут статьи не только primary category, но и cross-list category. Arxiv не умеет искать только по primary category.
search = arxiv.Search(
    query=f"cat:{CAT}",
    sort_by=arxiv.SortCriterion.SubmittedDate,
    max_results=N
)

In [6]:
# Скачиваем только те статьи у которых primary category - cs.CL
cnt = 0
for p in tqdm(client.results(search), desc='Скачивание статей...', total=N):
    if p.primary_category == CAT:
        p.download_source(dirpath=str(RAW_TEX_DIR)) # качаем TeX source
        p.download_pdf(dirpath=str(RAW_PDF_DIR)) # качаем pdf статьи
        cnt += 1
print(f'Скачано {cnt} статей с главной категорией {CAT}')

Скачивание статей...: 100%|██████████████████████████████████████████████████████████| 100/100 [00:54<00:00,  1.84it/s]

Скачано 74 статей с главной категорией cs.CL


После парсинга мы получаем в папке data директорию для каждой статьи, где хранятся LaTeX исходники и PDF документ

Также скорость скачивания довольно медленная. Сейчас не критично, но при масштабировании можно оптимизировать